In [1]:
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai

     |████████████████████████████████| 15.5 MB 790 kB/s eta 0:00:01
     |████████████████████████████████| 174 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 907 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 898 kB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 902 kB/s eta 0:00:01
     |████████████████████████████████| 177 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 337 kB 940 kB/s eta 0:00:01
     |████████████████████████████████| 64 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 322 kB/s eta 0:00:01
     |████████████████████████████████| 385 kB 477 kB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 1.5 MB/s eta 0:00:01    |█████████████████████████▌      | 9.0 MB 1.1 

In [2]:
%pip install llama-index

     |████████████████████████████████| 2.0 MB 464 kB/s eta 0:00:01
     |████████████████████████████████| 154 kB 612 kB/s eta 0:00:01
     |████████████████████████████████| 295 kB 555 kB/s eta 0:00:01
     |████████████████████████████████| 147 kB 604 kB/s eta 0:00:01
You should consider upgrading via the '/Users/nestorxyz/coding/learn/ai/learn-ai/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")